In [1]:
import nest_asyncio
nest_asyncio.apply()

In [60]:
import numpy as np
import pandas as pd
import stan
import arviz as az

In [3]:
import mmm_helpers as helpers

In [4]:
# Data    
# Four years' (209 weeks) records of sales, media impression and media spending at weekly level.   
df = pd.read_csv('data.csv')

In [5]:
# 1. media variables
# media impression
mdip_cols=[col for col in df.columns if 'mdip_' in col]
# media spending
mdsp_cols=[col for col in df.columns if 'mdsp_' in col]

In [6]:
# 2. control variables
# macro economics variables
me_cols = [col for col in df.columns if 'me_' in col]
# store count variables
st_cols = ['st_ct']
# markdown/discount variables
mrkdn_cols = [col for col in df.columns if 'mrkdn_' in col]
# holiday variables
hldy_cols = [col for col in df.columns if 'hldy_' in col]
# seasonality variables
seas_cols = [col for col in df.columns if 'seas_' in col]
base_vars = me_cols+st_cols+mrkdn_cols+hldy_cols+seas_cols

In [7]:
# 3. sales variables
sales_cols =['sales']

In [8]:
df_ctrl, sc_ctrl = helpers.mean_center_transform(df, ['sales']+me_cols+st_cols+mrkdn_cols)

In [9]:
df_ctrl = pd.concat([df_ctrl, df[hldy_cols+seas_cols]], axis=1)

In [10]:
df_ctrl.head()

,sales,me_ics_all,me_gas_dpg,st_ct,mrkdn_valadd_edw,mrkdn_pdm,hldy_Black Friday,hldy_Christmas Day,hldy_Christmas Eve,hldy_Columbus Day,...,seas_prd_12,seas_week_40,seas_week_41,seas_week_42,seas_week_43,seas_week_44,seas_week_45,seas_week_46,seas_week_47,seas_week_48
0,0.666808,0.878912,1.398726,1.086961,0.0,1.090623,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.729215,0.878912,1.393668,1.087178,0.0,1.069528,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.648481,0.878912,1.380828,1.088045,0.0,1.021879,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.635259,0.878912,1.374214,1.088045,0.0,1.057860,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.797662,0.901285,1.375770,1.088045,0.0,1.066801,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
len(df_ctrl), len(base_vars)

(209, 46)

In [55]:
ctrl_data = {
    # num observations
    'num_obs': len(df_ctrl),
    # num control vars
    'num_ctrl': len(base_vars),
    # control values (shape = num_obs,num_ctrl_vars)
    'ctrl_vals': df_ctrl[base_vars].values,
    # observed sales
    'y': df_ctrl['sales'].values,
}

In [56]:
ctrl_code = '''
data {
  // the total number of observations
  int num_obs;
  // the vector of sales
  real<lower=0> y[num_obs];
  int num_ctrl;
  // a matrix of control variables
  row_vector[num_ctrl] ctrl_vals[num_obs];
}

parameters {
  // residual variance
  real<lower=0> noise_var;
  // the intercept
  real tau;
  // coefficients for other control variables
  vector[num_ctrl] beta_ctrl;
}

transformed parameters {
  // a vector of the mean response
  real mu[num_obs];
  for (nn in 1:num_obs) {
    mu[nn] <- tau +
      dot_product(ctrl_vals[nn], beta_ctrl);
    }
}

model {
tau ~ normal(0, 5);
for (ctrl_index in 1:num_ctrl) {
beta_ctrl[ctrl_index] ~ normal(0,1);
}
noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
y ~ normal(mu, sqrt(noise_var));
}
'''

In [57]:
ctrl_posterior = stan.build(ctrl_code,data=ctrl_data,random_seed=1)

Building...

In file included from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun.hpp:110,
                 from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math/rev/fun.hpp:7,
                 from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /root/.cache/httpstan/4.5.0/models/d2jwa53r/model_d2jwa53r.cpp:2:
/home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun/generalized_inverse.hpp: In function ‘Eigen::Matrix<typename stan::value_type<T>::type, EigMat::ColsAtCompileTime, EigMat::RowsAtCompileTime> stan::math::generalized_inverse(const EigMat&)’:
/home/mike/venvs/mmm_venv/lib/python3.9/si


Building: 33.8s, done.Messages from stanc:
Warning in '/tmp/httpstan_xc21a965/model_d2jwa53r.stan', line 25, column 11: assignment operator <- is deprecated in the Stan language; use = instead.Warning in '/tmp/httpstan_xc21a965/model_d2jwa53r.stan', line 35, column 28: Argument 0.0005 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_xc21a965/model_d2jwa53r.stan', line 35, column 22: Argument 0.05 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).


In [58]:
ctrl_fit = ctrl_posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  30% (2400/8000)
Sampling:  31% (2500/8000)
Sampling:  32% (2600/8000)
Sampling:  34% (2700/8000)
Sampling:  35% (2800/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling:  40% (3200/8000)
Sampling:  41% (3300/8000)
Samplin

In [61]:
ctrl_inference = az.from_pystan(ctrl_fit)

In [65]:
ctrl_inference_summary = az.summary(ctrl_inference)

In [85]:
np.mean(ctrl_fit['beta_ctrl'],axis=1)

array([-0.12492236, -0.11532064,  0.1328281 ,  0.01920524,  0.67903357,
        0.17887454,  0.35008841, -0.49166239,  0.09303632,  0.50315812,
        0.38456832, -0.33490385,  0.00225207, -1.07696406, -0.0186507 ,
       -0.08674551, -0.23280091,  0.21433143, -0.02554093, -0.56603106,
        0.42613505,  0.16073853,  0.17135761,  0.13912171,  0.16462198,
        0.07893598, -0.10472506, -0.48945479, -0.32954603, -0.176919  ,
       -0.34341843, -0.36053307, -0.5629045 , -0.38688765, -0.32981113,
       -0.39536122, -0.45146543, -0.41357115, -0.11214163,  0.00727484,
        0.18531925,  0.47033923,  0.54778588,  1.7122181 ,  1.3999893 ,
       -0.15041996])

In [87]:
base_vars

['me_ics_all',
 'me_gas_dpg',
 'st_ct',
 'mrkdn_valadd_edw',
 'mrkdn_pdm',
 'hldy_Black Friday',
 'hldy_Christmas Day',
 'hldy_Christmas Eve',
 'hldy_Columbus Day',
 'hldy_Cyber Monday',
 'hldy_Day after Christmas',
 'hldy_Easter',
 "hldy_Father's Day",
 'hldy_Green Monday',
 'hldy_July 4th',
 'hldy_Labor Day',
 'hldy_MLK',
 'hldy_Memorial Day',
 "hldy_Mother's Day",
 'hldy_NYE',
 "hldy_New Year's Day",
 'hldy_Pre Thanksgiving',
 'hldy_Presidents Day',
 'hldy_Prime Day',
 'hldy_Thanksgiving',
 "hldy_Valentine's Day",
 'hldy_Veterans Day',
 'seas_prd_1',
 'seas_prd_2',
 'seas_prd_3',
 'seas_prd_4',
 'seas_prd_5',
 'seas_prd_6',
 'seas_prd_7',
 'seas_prd_8',
 'seas_prd_9',
 'seas_prd_12',
 'seas_week_40',
 'seas_week_41',
 'seas_week_42',
 'seas_week_43',
 'seas_week_44',
 'seas_week_45',
 'seas_week_46',
 'seas_week_47',
 'seas_week_48']

In [86]:
ctrl_inference_summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
noise_var,0.085,0.009,0.069,0.103,0.000,0.000,4782.0,3165.0,1.0
tau,0.650,0.945,-1.065,2.478,0.021,0.015,2125.0,2713.0,1.0
beta_ctrl[0],-0.125,0.618,-1.307,1.027,0.011,0.009,2911.0,2845.0,1.0
beta_ctrl[1],-0.115,0.255,-0.598,0.370,0.004,0.003,3309.0,2696.0,1.0
beta_ctrl[2],0.133,0.357,-0.509,0.825,0.006,0.005,3644.0,2971.0,1.0
...,...,...,...,...,...,...,...,...,...
mu[204],0.900,0.162,0.620,1.225,0.002,0.001,7155.0,3191.0,1.0
mu[205],0.860,0.152,0.571,1.137,0.002,0.001,6379.0,3655.0,1.0
mu[206],0.699,0.100,0.523,0.895,0.001,0.001,4958.0,3048.0,1.0
mu[207],0.688,0.102,0.499,0.884,0.001,0.001,4758.0,3413.0,1.0


In [21]:
# 2.2 Marketing Mix Model
df_mmm, sc_mmm = helpers.mean_log1p_transform(df, ['sales', 'base_sales'])
mu_mdip = df[mdip_cols].apply(np.mean, axis=0).values
max_lag = 8
num_media = len(mdip_cols)
# padding zero * (max_lag-1) rows
X_media = np.concatenate((np.zeros((max_lag-1, num_media)), df[mdip_cols].values), axis=0)
X_ctrl = df_mmm['base_sales'].values.reshape(len(df),1)
mmm_data = {
    'N': len(df),
    'max_lag': max_lag, 
    'num_media': num_media,
    'X_media': X_media, 
    'mu_mdip': mu_mdip,
    'num_ctrl': X_ctrl.shape[1],
    'X_ctrl': X_ctrl, 
    'y': df_mmm['sales'].values
}

In [22]:
mmm_code = '''
functions {
  // the adstock transformation with a vector of weights
  real Adstock(vector t, row_vector weights) {
    return dot_product(t, weights) / sum(weights);
  }
}
data {
  // the total number of observations
  int<lower=1> N;
  // the vector of sales
  real y[N];
  // the maximum duration of lag effect, in weeks
  int<lower=1> max_lag;
  // the number of media channels
  int<lower=1> num_media;
  // matrix of media variables
  matrix[N+max_lag-1, num_media] X_media;
  // vector of media variables' mean
  real mu_mdip[num_media];
  // the number of other control variables
  int<lower=1> num_ctrl;
  // a matrix of control variables
  matrix[N, num_ctrl] X_ctrl;
}
parameters {
  // residual variance
  real<lower=0> noise_var;
  // the intercept
  real tau;
  // the coefficients for media variables and base sales
  vector<lower=0>[num_media+num_ctrl] beta;
  // the decay and peak parameter for the adstock transformation of
  // each media
  vector<lower=0,upper=1>[num_media] decay;
  vector<lower=0,upper=ceil(max_lag/2.0)>[num_media] peak;
}
transformed parameters {
  // the cumulative media effect after adstock
  real cum_effect;
  // matrix of media variables after adstock
  matrix[N, num_media] X_media_adstocked;
  // matrix of all predictors
  matrix[N, num_media+num_ctrl] X;
  
  // adstock, mean-center, log1p transformation
  row_vector[max_lag] lag_weights;
  for (nn in 1:N) {
    for (media in 1 : num_media) {
      for (lag in 1 : max_lag) {
        lag_weights[max_lag-lag+1] <- pow(decay[media], (lag - 1 - peak[media]) ^ 2);
      }
     cum_effect <- Adstock(sub_col(X_media, nn, media, max_lag), lag_weights);
     X_media_adstocked[nn, media] <- log1p(cum_effect/mu_mdip[media]);
    }
  X <- append_col(X_media_adstocked, X_ctrl);
  } 
}
model {
  decay ~ beta(3,3);
  peak ~ uniform(0, ceil(max_lag/2.0));
  tau ~ normal(0, 5);
  for (i in 1 : num_media+num_ctrl) {
    beta[i] ~ normal(0, 1);
  }
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  y ~ normal(tau + X * beta, sqrt(noise_var));
}
'''

In [25]:
mmm_posterior = stan.build(mmm_code,data=mmm_data)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 51, column 35: assignment operator <- is deprecated in the Stan language; use = instead.
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 53, column 16: assignment operator <- is deprecated in the Stan language; use = instead.
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 54, column 34: assignment operator <- is deprecated in the Stan language; use = instead.
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 56, column 4: assignment operator <- is deprecated in the Stan language; use = instead.Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 66, column 30: Argument 0.0005 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 66, column 24: Argument 0.05 suggests there may be parameters that 

In [26]:
mmm_fit = mmm_posterior.sample(num_chains=3, num_samples=20)

Sampling:   0%
Sampling:   0% (1/3600)
Sampling:   0% (2/3600)
Sampling:   0% (3/3600)